# Numerical Techniques for the Wave Equation
## Math 437 Capstone Project, Fall 2023
### Author: Christian Lentz

The goal of this project will be to investigate numerical techniques for approximating solutions to PDEs. We will use finite differences to build a simple model for the wave equation with randomized initial conditions and periodic boundary conditions. We include four implementations of finite differences, including in both one and two spatial dimensions. This will motivate discussions and comparisons of other numerical techniques and thier implementations, particularly the Fourier spectral method. We include one implementation of the Fourier spectral method using the Fast Fourier Transform.

### 1  The Wave Equation

The wave equation is a second order partial differential equation which describes both traveling and standing waves as they occur in classical physics. The equation is particularly useful in modeling mechanical waves, such as sound waves, or electromagnetic waves, such as light waves. Note that the wave equation is not the same as the wave function, which is used in quantum mechanics to describe the quantum state of an isolated quantum system. 

Let $x$ be some $n$-dimensional vector $x = (x_1, x_2, \dots x_n)$, and suppose that $u(x,t)$ describes the displacement of a system from an initial rest position. The wave equation will describe the acceleration of this displacement over time. Specifically, the scalar form of the wave equation is

\begin{align*}
    \frac{\partial^2 u}{\partial t^2} &= c^2 (\frac{\partial^2 u}{\partial x_{1}^2} + \dots + \frac{\partial^2 u}{\partial x_{n}^2})
\end{align*}, 

where $c \in \mathbb{R^{>0}}$ is some scalar. The vectorized form can be written as 

\begin{align*}
    \frac{\partial^2 u}{\partial t^2} &= c^2 {\nabla}^2 u 
\end{align*}

This form of the wave equation which we use assumes no friction. For this project, we will use finite difference and spectral methods to approximate solutions to the wave equation with periodic boundary conditons in order to investigate and compre the underlying numerical techniques. Throghout, we discuss useful modeling applications.   

### 2  Finite Differences

The finite differences method makes use of the definition of the derivative, or the difference quotient. However, rather than taking the limit, we will approximate solutions to the PDE at finitely many fixed points in time, hence why we call this finite differences. 

![Multiple solutions to the 1D wave equation with Finite Differences](visuals/1D_progress/FD_1D_5.png)

#### 2.1  A bit of Analysis

First, we recall the definition of the derivative. Suppose some function $f: E \rightarrow \mathbb{R}$ where $x_{0} \in E$ is a limit point of $E \subseteq \mathbb{R}$. We say that $f$ is differentiable at $x_{0}$ if the following limit exists: 

\begin{align*}
    lim_{x \rightarrow 0} &= \frac{f(x_0 + x)-f(x_0)}{x} 
\end{align*}

Equivalently, we may also write the difference quotient as: 

\begin{align*}
    lim_{x \rightarrow 0} &= \frac{f(x_0)-f(x_0 - x)}{x} 
\end{align*}

Writing the difference quotient in either of these forms will be useful in proving the difference quotient for the second derivative of a function. To make our definition above more concise, it is also useful to recall that if $x_0 \in E$ is a limit point, then: 

\begin{align*}
    \forall \epsilon > 0: \exists x \in E: 0 < |x - x_0| < \epsilon 
\end{align*}

In words, $E$ contains points which are arbitrarily close but not equal to $x_0$. This definition is the foundation of the finite differences approach. By manipulating the difference quotient, we can build a technique to approximte solutions to any ODE or PDE of any order. It is possible to do this since we can generalize the definition of the difference quotient to higher order derivatives as well. For our purpose, we will need to know the difference quotient for the second derivative of a function. 

$\textit{Claim:}$ Let $f: \mathbb{R} \rightarrow \mathbb{R}$ be differentiable and let $x_0 \in \mathbb{R}$. Further, suppose that $f''(x_0)$ exists. Then:

\begin{align*}
    f''(x_0) &= lim_{x \rightarrow 0} \frac{f(x_0 + x) + f(x_0 - x) - 2f(x_0)}{x^2}
\end{align*}

We can provide two proofs of this fact. The first is quite clear but is also quite and abuse of notation. The second makes use of the Cauchy Mean Value Theorem. 

$\textit{Proof 1:}$ Since $f''(x_0)$ exists, it follows that: 

\begin{align*}
    f''(x_0) &= lim_{x \rightarrow 0} \frac{f'(x_0 + x) - f'(x_0)}{x} 
\end{align*}

Further, since $f$ is a differentiable function, then we have:

\begin{align*}
    f''(x_0) &= lim_{x \rightarrow 0} \frac{\frac{f(x_0 + x_1) - f(x_0)}{x_1} - \frac{f(x_0) - f(x_0 - x_2)}{x_2}}{x} \\
\end{align*}

Letting $x = x_1 = x_2$, then we may simplify to get the form stated in the claim, and this concludes the proof. 

$\textit{Proof 2:}$

ADD THIS PROOF!!!

#### 2.2  Finite Differences for the Wave Equation

Now that we have defined the difference quotient for the first and second derivative of a function, we can use these facts to build a method for approximating solutions to first or second order PDEs. Here, we outline our technique for the wave equation, and in the following sections we inclulde an example of finite differences code for an advection-diffusion PDE. 

Consider the wave equation in two spatial dimensions $x$ and $y$: 

\begin{align*}
    \frac{\partial^2 u}{\partial t^2} &= c^2 (\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2})
\end{align*}

The first step of this process will be to write the PDE using the difference quotient. To make things a bit more intuitive, we will write the limits as $lim_{\Delta t \rightarrow 0}$, $lim_{\Delta x \rightarrow 0}$ and $lim_{\Delta y \rightarrow 0}$. Suppose that we let $x = a$, $y = b$ and $t = T$. Then we have:  

\begin{align*}
    lim_{\Delta t \rightarrow 0} \frac{u(a, b, T + \Delta t) + u(a, b, T - \Delta t) - 2u(a, b, T)}{(\Delta t)^2} = \dots\\
    \dots = c^2(lim_{\Delta x \rightarrow 0} \frac{u(a + \Delta x, b, T) + u(a - \Delta x, b, T) - 2u(a, b, T)}{(\Delta x)^2} + \dots \\
    \dots + lim_{\Delta y \rightarrow 0} \frac{u(a, b + \Delta y, T) + u(a, b - \Delta y, T) - 2u(a, b, T)}{(\Delta y)^2}) 
\end{align*}

By inspection, we see that this is quite easy to generalize to the $n$-dimensional case. Although this form is quite cumbersome, we can leverage it to numerically approximate solutions to the wave equation. The key is to ignore the limit, and let $u(a,b,T+\Delta t)$ be unknown. By doing this, we will be able to approximate solutions for fixed $t$ beyond our initial time $T$. Thus, we get something that looks like this:

\begin{align*}
    u(a, b, T + \Delta t) \approx \dots \\
    \dots \approx 2u(a,b,T) - u(a, b, T - \Delta t) + \dots \\ 
    \dots + \frac{c^2 (\Delta t)^2}{(\Delta x)^2} (u(a + \Delta x, b, T) + u(a - \Delta x, b, T) - 2u(a, b, T)) + \dots \\ 
    \dots + \frac{c^2 (\Delta t)^2}{(\Delta y)^2} (u(a, b + \Delta y, T) + u(a, b - \Delta y, T) - 2u(a, b, T)) 
\end{align*}

Using this equation, we can approximate solutions for our PDE. 

#### 3  Choosing $\Delta x$, $\Delta y$, and $\Delta t$

We must take care when chooisng parameter values for $\Delta x$, $\Delta y$, and $\Delta t$ to assure the stability of our program. In certain cases, a poor choice for these parameters will result in our program producing incorrect solutions.

Suppose our wave PDE with solutions of the form $u(x, y, t)$ where $x,y \in R$ and $t \in \mathbb{R}^{>0}$. Such a PDE can be thought of as an uncountably infinite function space, where at each $t \in \mathbb{R}^{>0}$ we have a unique function describing a solution to the PDE. Numerical techniques for approximating solutions to such a PDE, including finite differences, will rely on a $\textbf{discretization}$ of this infinite-dimensional function space to an sequence of finite-dimensional probelms.

Suppose that we wish to model our PDE over the time interval T = $[t_0, t_{max}]$. A discretization of the time variable will split this iterval into finitely many points by choosing some $\Delta t$. The cardinality of this disretized set is: 

\begin{align*}
    N = \frac{t_{max} - t_0}{\Delta t} 
\end{align*}

And the set itself is 

\begin{align*}
    dT = \{ t_0 + \Delta t * i \} \text{ for } i \in \{1, \dots, N\}
\end{align*}

By doing this, we have transformed the uncountably infinite interval $T$ into a finite sequence of $N$ probelms, where we approximate a solution to the PDE for each $t \in dT$. We can easily generalize this to the case where we wish to model the PDE for all $t \in \mathbb{R}^{>0}$ by simply letting $t_0 = 0$ and $t_{max} = \infty$. In this case, $dT$ is a countably infinite set. However, we must also place a discretization on the spatial variables as well. Thus, we must also choose some $\Delta x$ and $\Delta y$ in order to transform the space into a discretized grid. 

At a high level, we must choose these parameters such that $\Delta t$ is not too large as too allow the wave to travel to another coordinate in the grid within that interval of time. We can ensure proper selection of these parameters by adhering to the $\textbf{CFL condition}$, which is commonly used with finite differences to model advection like processes. The condition states that for a PDE with spatial variables $x = (x_1, \dots, x_n)$ and temporal variable $t$, a discretization of these parameters must satisfy that 

\begin{align*}
    \Delta t (\sum_{i=1}^{n} \frac{u_{x_i}}{\Delta x_i}) \leq C
\end{align*}

where $C$ is a dimensionless constant and $u_{x_i}$ is the velocity with respect to $x_i$. We typically let $C = 1$. Essentially, the CFL conditions tells us that if we choose a small value for $\Delta x$ and $\Delta y$, we must also choose a sufficiently small value for $\Delta t$ as well. 

#### 4  Implementation

DESCRIBE HOW WE IMPLEMENT FD IN THIS PROJECT 



### 5  Other Numerical Methods

It is worth noting other common classes of numerical techniques, including Finite Element Methods (FEMs) and Finite Volume Methods. FEMs and Finite Elements both provide continuous or discontinuous piecewise approximations over a tesselation of the domain. While they are more similar to Finite Differences than Spectral Methods, they are considerably more difficult mathematically when compared to finite differences, and are also more difficult to implement. However, $\textbf{Spectral Methods}$ can deliver highly accurate, $\textbf{smooth}$ approximations for a solution to a PDE.  

#### 5.1 Spectral Methods: A Brief Overview 

Spectral methods approximate solutions to PDEs in the form of fixed degree polynomials, which are $\textbf{smooth}$ by definiton. Recall that for some open set $U$, any function $f: U \rightarrow \mathbb{R}$ is said to belong to the $\textbf{differentiability class } C^n$ iff each higher-order derivative of $f$ in the set $\{f^{(i)}\}_{i=1}^{n}$ exists and is continuous over $U$. A smooth function belongs to the class $C^{\infty}$, or is $\textbf{infinitely differentiable}$. If the underlying PDE is a smooth function, then a spectral method can provide a highly accurate approximation. 

More specifically, Spectral Methods will seek an approximation in the form of a linear combination of a set of mutually $\textbf{orthogonal polynomials}$. Suppose the following linear combination of polynomials over the reals:

\begin{align*}
    \sum_{i=1}^{n} f_i(x)
\end{align*}

For any two polynomials $f_j(x)$ and $f_k(x)$ for $j,k \in \{1, \dots, n\}$, we may define the inner product

\begin{align*}
    <f_j,f_k> &= \int f_j(x) f_k(x) d \alpha(x) 
\end{align*}

where $\alpha$ is a non-decreasing function over the reals. If $<f_j,f_k>$ = 0, then the polynomials are said to be orthogonal. Thus, a spectral method will seek to construct a space of mutually orthogonal functions which we use to approximate the PDE. We will not provide a full discussion of spectral methods in the general sense, but instead will discuss within the context of the Fourier method for the wave equation in one spatial dimension.   

### 6  Fourier Analysis of the Wave Equation

Fourier analysis is an extension of the study of Fourier series, in which we wish to represent some function as a linear combinations of trigonomentric functions. The key insight is that we can express any general function in this way. Fourier analysis has applications across pure and applied mathemaitcs, including numerical techniques for PDEs. First, we examine the wave equation using these techniques, and use this to show that we can construct a spectral method using the $\textbf{Fourier Transform}$. 

Consider the wave equation in one spatial dimension: 

\begin{align}
    u_{tt} &= c^2 u_{xx}
\end{align} 
    
Suppose that we wish to use ODE technques to solve, and that we try to solve using the following ansatz: 

\begin{align*}
    u_1(x,t) &= A(t)coskx 
\end{align*}

By plugging in and solving in the usual way (which we do not explicitly show here), we can show that $u_1$ is in fact a solution, but only under the assumption that $A(t)$ is a $\textbf{harmonic oscillator}$. Equivalently, using Newton's Second Law, we can say that $A$ satisfies that

\begin{align*}
    A_{tt} &= -k^2 v^2 A(t)
\end{align*}

Following the same logic, we can also say that 

\begin{align*}
    u_2(x,t) &= B(t)sinkx
\end{align*}

is a general solution, under the assumption that $B(t)$ is also a harmonic oscillator. In fact it, is also quite easy to show that if $u_1$ and $u_2$ are solutions to (1), then so too is $u_1 + u_2$. Therefore, we may actually write our solution to the wave equation using arbitrary linear combinations of $u_1$ and $u_2$ with varying wavelengths $k$.  

\begin{align*}
    u(x,t) &= A_1(t)cosk_1 x + B_1(t)sink_1 x + A_2(t)cosk_2 x + B_2(t)sink_2 x + ...
\end{align*}

Such a solution works in a discrete setting, where the underlying function that we are approximating solutions for is periodic, and we have a sequence of equally spaced values $k_n$. To bring this into a more general setting, we must work with a continuum of values $k$, such that $\Delta k \rightarrow 0$ as $k \rightarrow \infty$. Thus, rather than using a sum, we must integrate: 

\begin{align*}
    u(x,t) &= \int_0^{\infty} A(k,t)coskx + B(k,t)sinkx dk
\end{align*}

Recall that since $A(t)$ and $B(t)$ are harmonic oscillators, and thus rely on $k$, we must now make explicit that both $A$ and $B$ are actually functions of $k$, since $k$ now ranges over a continuum of values. Thanks to Euler, we may write this in a more concise form: 

\begin{align*}
    u(x,t) &= \int_{-\infty}^{\infty} u(k,t) e^{ikx} dk
\end{align*}

where $A = 2Re(u)$ and $B = 2Im(u)$. What we have just derived is the $\textbf{Inverse Fourier}$ $\textbf{Transform}$, which describes how we may write $u(x,t)$ as a sum over a continuum of its frequencies. The $\textbf{Fourier Transform}$ is the integral transform which actually sends $u(x,t)$ into the form $u(k,t)$: 

\begin{align*}
    u(k,t) &= \int_{-\infty}^{\infty} u(x,t) e^{-ikx} dx
\end{align*}

### 7  The Fourier Spectral Method 

We have shown that we can write the wave equation as a sum of sine and cosine functions of varying frequeinces using the Fourier transform. Recall that we previously noted that a spectral method for approximating solutions to PDEs will seek solutions in the form of linear combinations of mutually orthogonal polynomials. In fact, it is quite easy to show that sine and cosine functions of arbitrary frequency are indeed orthogonal over correctly chosen bounds. Let $N \in \mathbb{R}^{>0}$ and $m,n \in \mathbb{R}$. Using the inner product for polynomials, it follows that

\begin{align*}
    \int_{-N}^{N} cos(mx)sin(nx) dx &= 0, 
\end{align*}

thus implying that we may use the Fourier transform to construct a spectral method for PDEs. In general, the Fourier spectral method will also place a discretization on the function space. This means that we must use a discretized verison of the Fourier transfrom. 

#### 7.3  The Discrete Fourier Transform

#### 7.4  Fast Fourier Trasnform 

UNDERLYING ALGORITHM FOR THIS IMPLEMENTATION! 
    - discuss motivation for this 
    - discuss other applications

#### 7.5  Fourier Spectral Method for the Wave Equation

THIS OUTILNES WHAT OUR CODE WILL LOOK LIKE! 

### 8  Conclusions

### 9  References 